# DataLab Cup 3: Reverse Image Caption

# Download Dataset from Kaggle

In [ ]:
# Follow the tutorial in this page:
# https://www.endtoend.ai/tutorial/how-to-download-kaggle-datasets-on-ubuntu/
!cat ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c datalab-cup3-reverse-image-caption-2021

In [ ]:
import os
import zipfile

# zipfile example
def zip_list(file_path):
    zf = zipfile.ZipFile(file_path, 'r')
    zf.extractall('./')

file_path = './datalab-cup3-reverse-image-caption-2021.zip'
zip_list(file_path)

# Libraries and Packages Import

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import layers
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable warnings, info and errors 
import string
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import random
import time
from pathlib import Path
from tqdm import tqdm

import re
from IPython import display

In [ ]:
!nvidia-smi

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        #for gpu in gpus:
        #    tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Preprocessing

## Preprocess text

In [ ]:
dictionary_path = './dictionary'
vocab = np.load(dictionary_path + '/vocab.npy')
print('there are {} vocabularies in total'.format(len(vocab)))

word2Id_dict = dict(np.load(dictionary_path + '/word2Id.npy'))
id2word_dict = dict(np.load(dictionary_path + '/id2Word.npy'))
print('Word to id mapping, for example: %s -> %s' % ('flower', word2Id_dict['flower']))
print('Id to word mapping, for example: %s -> %s' % ('1', id2word_dict['1']))
print('Tokens: <PAD>: %s; <RARE>: %s' % (word2Id_dict['<PAD>'], word2Id_dict['<RARE>']))

In [ ]:
def sent2IdList(line, MAX_SEQ_LENGTH=20):
    MAX_SEQ_LIMIT = MAX_SEQ_LENGTH
    padding = 0
    
    # data preprocessing, remove all puntuation in the texts
    prep_line = re.sub('[%s]' % re.escape(string.punctuation), ' ', line.rstrip())
    prep_line = prep_line.replace('-', ' ')
    prep_line = prep_line.replace('-', ' ')
    prep_line = prep_line.replace('  ', ' ')
    prep_line = prep_line.replace('.', '')
    tokens = prep_line.split(' ')
    tokens = [
        tokens[i] for i in range(len(tokens))
        if tokens[i] != ' ' and tokens[i] != ''
    ]
    l = len(tokens)
    padding = MAX_SEQ_LIMIT - l
    
    # make sure length of each text is equal to MAX_SEQ_LENGTH, and replace the less common word with <RARE> token
    for i in range(padding):
        tokens.append('<PAD>')
    line = [
        word2Id_dict[tokens[k]]
        if tokens[k] in word2Id_dict else word2Id_dict['<RARE>']
        for k in range(len(tokens))
    ]

    return line

text = "the flower shown has yellow anther red pistil and bright red petals."
print(text)
print(sent2IdList(text))

## Dataset

In [ ]:
data_path = './dataset'
df = pd.read_pickle(data_path + '/text2ImgData.pkl')
num_training_sample = len(df)
n_images_train = num_training_sample
print('There are %d image in training data' % (n_images_train))

In [ ]:
df.head(5)

# Data Augmentation

In [31]:
# in this competition, you have to generate image in size 64x64x3
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_CHANNEL = 3
MAPPING_FUNC = None

def vertical_flip(tf_img):
    return tf.image.flip_left_right(tf_img)

def horizontal_flip(tf_img):
    return tf.image.flip_up_down(tf_img)

def brightness(tf_img):
    return tf.image.random_brightness(tf_img, 0.2, 2)

def data_generator(caption, image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.per_image_standardization(img)
    img.set_shape([None, None, 3])
    
    if MAPPING_FUNC is not None:
        img = MAPPING_FUNC(img)

    img = tf.image.resize(img, size=[IMAGE_HEIGHT, IMAGE_WIDTH])
    img.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNEL])
    caption = tf.cast(caption, tf.int32)    
    return img, caption

def transform_dataset(caption, image_path):
    dataset = tf.data.Dataset.from_tensor_slices((caption, image_path))
    dataset = dataset.map(data_generator, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(len(caption))
    return dataset

def dataset_generator(filenames, batch_size):
    # load the training data into two NumPy arrays
    df = pd.read_pickle(filenames)
    captions = df['Captions'].values
    caption = []
    # each image has 1 to 10 corresponding captions
    # we choose one of them randomly for training
    for i in range(len(captions)):
        caption.append(random.choice(captions[i]))
    caption = np.asarray(caption)
    caption = caption.astype(np.int64)
    image_path = df['ImagePath'].values
    
    # assume that each row of features corresponds to the same row as labels.
    assert caption.shape[0] == image_path.shape[0]
    
    original_dataset = transform_dataset(caption, image_path)

    MAPPING_FUNC = vertical_flip
    vertical_dataset = transform_dataset(caption, image_path)

    MAPPING_FUNC = horizontal_flip
    horizontal_dataset = transform_dataset(caption, image_path)

    MAPPING_FUNC = brightness
    brightness_dataset = transform_dataset(caption, image_path)
    
    dataset = original_dataset.concatenate(vertical_dataset)
    dataset = dataset.concatenate(horizontal_dataset)
    dataset = dataset.concatenate(brightness_dataset)
    dataset = dataset.shuffle(len(caption)).batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [32]:
BATCH_SIZE = 64
dataset = dataset_generator(data_path + '/text2ImgData.pkl', BATCH_SIZE)

# Hyperparameters

In [33]:
hparas = {
    'MAX_SEQ_LENGTH': 20,                     # maximum sequence length
    'EMBED_DIM': 256,                         # word embedding dimension
    'VOCAB_SIZE': len(word2Id_dict),          # size of dictionary of captions
    'RNN_HIDDEN_SIZE': 128,                   # number of RNN neurons
    'Z_DIM': 512,                             # random noise z dimension
    'DENSE_DIM': 128,                         # number of neurons in dense layer
    'IMAGE_SIZE': [64, 64, 3],                # render image size
    'BATCH_SIZE': 64,
    'LR': 1e-4,
    'LR_DECAY': 0.5,
    'BETA_1': 0.5,
    'N_EPOCH': 1000,
    'N_SAMPLE': num_training_sample,          # size of training data
    'CHECKPOINTS_DIR': './checkpoints/demo',  # checkpoint path
    'PRINT_FREQ': 1                           # printing frequency of loss
}

# Conditioinal GAN Model

## Text Encoder

In [34]:
class TextEncoder(tf.keras.Model):
    """
    Encode text (a caption) into hidden representation
    input: text, which is a list of ids
    output: embedding, or hidden representation of input text in dimension of RNN_HIDDEN_SIZE
    """
    def __init__(self, hparas):
        super(TextEncoder, self).__init__()
        self.hparas = hparas
        self.batch_size = self.hparas['BATCH_SIZE']
        
        # embedding with tensorflow API
        self.embedding = layers.Embedding(self.hparas['VOCAB_SIZE'], self.hparas['EMBED_DIM'])
        # RNN, here we use GRU cell, another common RNN cell similar to LSTM
        self.gru = layers.GRU(self.hparas['RNN_HIDDEN_SIZE'],
                              return_sequences=True,
                              return_state=True,
                              recurrent_initializer='glorot_uniform')

        # self.bidirect = layers.Bidirectional(layers.LSTM(128))
        # self.d1 = layers.Dense(128, activation="relu")
    
    def call(self, text, hidden):
        text = self.embedding(text)
        output, state = self.gru(text, initial_state = hidden)
        # output = self.bidirect(text)
        # output = self.d1(output)
        return output[:, -1, :], state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.hparas['BATCH_SIZE'], self.hparas['RNN_HIDDEN_SIZE']))

## Generator

In [35]:
class Generator(tf.keras.Model):
    """
    Generate fake image based on given text(hidden representation) and noise z
    input: text and noise
    output: fake image with size 64*64*3
    """
    def __init__(self, hparas):
        super(Generator, self).__init__()
        self.hparas = hparas
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(self.hparas['DENSE_DIM'])
        self.d2 = tf.keras.layers.Dense(64*64*3)

        self.d3 = layers.Dense(16*16*256, use_bias=False)
        self.bn1= layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()

        self.Conv2DTrans1 = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding="same", use_bias=False)
        self.Conv2DTrans2 = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding="same", use_bias=False)
        self.Conv2DTrans3 = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding="same", use_bias=False)
   
    def call(self, text, noise_z):
        text = self.flatten(text)
        text = self.d1(text)
        text = tf.nn.leaky_relu(text)
        
        # concatenate input text and random noise",
        text_concat = tf.concat([noise_z, text], axis=1)
        text_concat = self.d3(text_concat)
        text_concat = self.bn1(text_concat)
        text_concat = tf.reshape(text_concat, [-1, 16, 16, 256])
        
        text_concat = self.Conv2DTrans1(text_concat)
        text_concat = self.bn2(text_concat)
        text_concat = tf.nn.leaky_relu(text_concat)

        text_concat = self.Conv2DTrans2(text_concat)
        text_concat = self.bn3(text_concat)
        text_concat = tf.nn.leaky_relu(text_concat)

        text_concat = self.Conv2DTrans3(text_concat)

        logits = tf.reshape(text_concat, [-1, 64, 64, 3])
        output = tf.nn.tanh(logits)

        return logits, output

## Discriminator

In [36]:
class Discriminator(tf.keras.Model):
    """
    Differentiate the real and fake image
    input: image and corresponding text
    output: labels, the real image should be 1, while the fake should be 0
    """
    def __init__(self, hparas):
        super(Discriminator, self).__init__()
        self.hparas = hparas
        self.flatten = tf.keras.layers.Flatten()
        self.d_text = tf.keras.layers.Dense(self.hparas['DENSE_DIM'])
        self.d_img = tf.keras.layers.Dense(self.hparas['DENSE_DIM'])
        self.d = tf.keras.layers.Dense(1)

       
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()
        self.dropout = layers.Dropout(0.3)

        self.Conv2D1 = layers.Conv2D(64, (5, 5), strides=(2, 2), padding="same")
        self.Conv2D2 = layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same")

    def call(self, img, text):
        text = self.flatten(text)
        text = self.d_text(text)
        text = self.bn1(text)
        text = tf.nn.leaky_relu(text)

        img = self.Conv2D1(img)
        img = self.bn2(img)
        img = tf.nn.leaky_relu(img)
        img = self.dropout(img)

        img = self.Conv2D2(img)
        img = self.bn3(img)
        img = tf.nn.leaky_relu(img)
        img = self.dropout(img)
        img = self.flatten(img)
        img = self.d_img(img)
        
        # concatenate image with paired text
        img_text = tf.concat([text, img], axis=1)
        logits = self.d(img_text)
        output = tf.nn.sigmoid(logits)
        
        return logits, output

In [37]:
text_encoder = TextEncoder(hparas)
generator = Generator(hparas)
discriminator = Discriminator(hparas)

## Training Settings

In [38]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [53]:
def discriminator_loss(real_logits, fake_logits):
    # output value of real image should be 1
    real_loss = cross_entropy(tf.ones_like(real_logits), real_logits)
    # output value of fake image should be 0
    fake_loss = cross_entropy(tf.zeros_like(fake_logits), fake_logits)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # output value of fake image should be 0
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [54]:
# we use seperated optimizers for training generator and discriminator
generator_optimizer = tf.keras.optimizers.Adam(hparas['LR'])
discriminator_optimizer = tf.keras.optimizers.SGD(hparas['LR'])

In [55]:
# one benefit of tf.train.Checkpoint() API is we can save everything seperately
checkpoint_dir = hparas['CHECKPOINTS_DIR']
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 text_encoder=text_encoder,
                                 generator=generator,
                                 discriminator=discriminator)

In [56]:
@tf.function
def DC_DTrain(image, caption, hidden, noise):
    # z = tf.random.normal(hparas['BZ'])
    noise_g = tf.random.normal(shape=[hparas['BATCH_SIZE'], hparas['Z_DIM']], mean=0.0, stddev=1.0)

    with tf.GradientTape() as tp:
        with tf.GradientTape() as tp_gp:
            text_embed, hidden = text_encoder(caption, hidden)
            # _, fake_image = generator(text_embed, noise)
            # real_logits, real_output = discriminator(image, text_embed)
            # fake_logits, fake_output = discriminator(fake_image, text_embed)

            _, x_bar = generator(text_embed, noise_g, training = True)
            epsilon = tf.random.uniform([BATCH_SIZE,1,1,1])
            x = image
            x_hat = epsilon * x + (1. - epsilon) * x_bar

            x_bar = x_bar + noise * tf.random.normal(x_bar.shape)
            x = x + noise * tf.random.normal(x.shape)
            x_hat = x_hat + noise * tf.random.normal(x_hat.shape)

            z0, _ = discriminator(x_bar, text_embed, training = True)
            z1, _ = discriminator(x, text_embed, training = True)
            z2, _ = discriminator(x_hat, text_embed, training = True)

            gradient_penalty = tp_gp.gradient(z2,x_hat)
            gradient_penalty = tf.sqrt(tf.reduce_sum(tf.math.square(gradient_penalty),axis=[1,2,3]))
            loss = z0 - z1 + 10. * tf.math.square((gradient_penalty - 1.))
            ld = tf.reduce_mean(loss)
            lg = - tf.reduce_mean(z0)

    gradient_d = tp.gradient(ld, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradient_d, discriminator.trainable_variables))

    return lg, ld

@tf.function
def DC_GTrain(image, caption, hidden, noise):
    # z = tf.random.normal(hparas['BZ'])
    noise_g = tf.random.normal(shape=[hparas['BATCH_SIZE'], hparas['Z_DIM']], mean=0.0, stddev=1.0)

    with tf.GradientTape() as tp:
        with tf.GradientTape() as tp_gp:
            text_embed, hidden = text_encoder(caption, hidden)
            # _, fake_image = generator(text_embed, noise)
            # real_logits, real_output = discriminator(image, text_embed)
            # fake_logits, fake_output = discriminator(fake_image, text_embed)

            _, x_bar = generator(text_embed, noise_g, training = True)
            epsilon = tf.random.uniform([BATCH_SIZE,1,1,1])
            x = image
            x_hat = epsilon * x + (1. - epsilon) * x_bar

            x_bar = x_bar + noise * tf.random.normal(x_bar.shape)
            x = x + noise * tf.random.normal(x.shape)
            x_hat = x_hat + noise * tf.random.normal(x_hat.shape)

            z0, _ = discriminator(x_bar, text_embed, training = True)
            z1, _ = discriminator(x, text_embed, training = True)
            z2, _ = discriminator(x_hat, text_embed, training = True)

            gradient_penalty = tp_gp.gradient(z2,x_hat)
            gradient_penalty = tf.sqrt(tf.reduce_sum(tf.math.square(gradient_penalty),axis=[1,2,3]))
            loss = z0 - z1 + 10. * tf.math.square((gradient_penalty - 1.))
            ld = tf.reduce_mean(loss)
            lg = - tf.reduce_mean(z0)

    gradient_g = tp.gradient(lg, generator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradient_g, generator.trainable_variables))

    return lg, ld

In [57]:
@tf.function
def test_step(caption, noise, hidden):
    text_embed, hidden = text_encoder(caption, hidden)
    _, fake_image = generator(text_embed, noise)
    return fake_image

In [58]:
DC_Train = (
    DC_DTrain,
    DC_DTrain,
    DC_DTrain,
    DC_DTrain,
    DC_DTrain,
    DC_GTrain,
)

DC_Critic = len(DC_Train)

## Visulization

In [59]:
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

def imsave(images, size, path):
    # getting the pixel values between [0, 1] to save it
    return plt.imsave(path, merge(images, size)*0.5 + 0.5)

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

In [60]:
def sample_generator(caption, batch_size):
    caption = np.asarray(caption)
    caption = caption.astype(np.int64)
    dataset = tf.data.Dataset.from_tensor_slices(caption)
    dataset = dataset.batch(batch_size)
    return dataset

In [61]:
ni = int(np.ceil(np.sqrt(hparas['BATCH_SIZE'])))
sample_size = hparas['BATCH_SIZE']
sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, hparas['Z_DIM'])).astype(np.float32)
sample_sentence = ["the flower shown has yellow anther red pistil and bright red petals."] * int(sample_size/ni) + \
                  ["this flower has petals that are yellow, white and purple and has dark lines"] * int(sample_size/ni) + \
                  ["the petals on this flower are white with a yellow center"] * int(sample_size/ni) + \
                  ["this flower has a lot of small round pink petals."] * int(sample_size/ni) + \
                  ["this flower is orange in color, and has petals that are ruffled and rounded."] * int(sample_size/ni) + \
                  ["the flower has yellow petals and the center of it is brown."] * int(sample_size/ni) + \
                  ["this flower has petals that are blue and white."] * int(sample_size/ni) +\
                  ["these white flowers have petals that start off white in color and end in a white towards the tips."] * int(sample_size/ni)

for i, sent in enumerate(sample_sentence):
    sample_sentence[i] = sent2IdList(sent)
sample_sentence = sample_generator(sample_sentence, hparas['BATCH_SIZE'])

# Training

In [62]:
if not os.path.exists('samples/demo'):
    os.makedirs('samples/demo')

In [63]:
def train(dataset, epochs):
    # hidden state of RNN
    hidden = text_encoder.initialize_hidden_state()
    steps_per_epoch = int(hparas['N_SAMPLE']/hparas['BATCH_SIZE'])
    
    ctr = 0
    for epoch in range(hparas['N_EPOCH']):
        g_total_loss = 0
        d_total_loss = 0
        start = time.time()
        
        if epoch < 200:
            noise = 1.0 / float(epoch + 1)
        else:
            noise = 0.0
        
        for image, caption in dataset:
            lg, ld = DC_Train[ctr](image, caption, hidden, noise) # (real_img, embed, noise_decay)
            ctr += 1
            g_total_loss += lg.numpy()
            d_total_loss += ld.numpy()
            if ctr == DC_Critic : ctr = 0
            
        time_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", time_tuple)
            
        print("Epoch {}, gen_loss: {:.4f}, disc_loss: {:.4f}".format(epoch+1,
                                                                     g_total_loss/steps_per_epoch,
                                                                     d_total_loss/steps_per_epoch))
        print('Time for epoch {} is {:.4f} sec'.format(epoch+1, time.time()-start))
        
        # save the model
        if (epoch + 1) % 50 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        
        # visualization
        if (epoch + 1) % hparas['PRINT_FREQ'] == 0:
            for caption in sample_sentence:
                fake_image = test_step(caption, sample_seed, hidden)
            save_images(fake_image, [ni, ni], 'samples/demo/train_{:02d}.jpg'.format(epoch))

In [64]:
train(dataset, hparas['N_EPOCH'])

Epoch 1, gen_loss: 313.9078, disc_loss: -116.1090
Time for epoch 1 is 75.0370 sec
Epoch 2, gen_loss: 276.9704, disc_loss: -81.0076
Time for epoch 2 is 73.3980 sec
Epoch 3, gen_loss: 213.6130, disc_loss: -66.5658
Time for epoch 3 is 73.5407 sec
Epoch 4, gen_loss: 155.3447, disc_loss: -53.8502
Time for epoch 4 is 73.5087 sec
Epoch 5, gen_loss: 119.8926, disc_loss: -43.3663
Time for epoch 5 is 73.4699 sec
Epoch 6, gen_loss: 74.4416, disc_loss: -32.9161
Time for epoch 6 is 73.4444 sec
Epoch 7, gen_loss: 66.3515, disc_loss: -33.3799
Time for epoch 7 is 73.5760 sec
Epoch 8, gen_loss: 64.0866, disc_loss: -34.2756
Time for epoch 8 is 73.4445 sec
Epoch 9, gen_loss: 71.0477, disc_loss: -40.4738
Time for epoch 9 is 73.2564 sec
Epoch 10, gen_loss: 98.0182, disc_loss: -48.5343
Time for epoch 10 is 73.5432 sec
Epoch 11, gen_loss: 112.0178, disc_loss: -44.1488
Time for epoch 11 is 73.6910 sec
Epoch 12, gen_loss: 117.9820, disc_loss: -43.2097
Time for epoch 12 is 73.4062 sec
Epoch 13, gen_loss: 125.74

In [ ]:
print(len(dataset))

# Evaluation

## Testing Dataset
If you change anything during preprocessing of training dataset, you must make sure same operations have be done in testing dataset.

In [ ]:
def testing_data_generator(caption, index):
    caption = tf.cast(caption, tf.float32)
    return caption, index

def testing_dataset_generator(batch_size, data_generator):
    data = pd.read_pickle('./dataset/testData.pkl')
    captions = data['Captions'].values
    caption = []
    for i in range(len(captions)):
        caption.append(captions[i])
    caption = np.asarray(caption)
    caption = caption.astype(np.int64)
    index = data['ID'].values
    index = np.asarray(index)
    
    dataset = tf.data.Dataset.from_tensor_slices((caption, index))
    dataset = dataset.map(data_generator, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat().batch(batch_size)
    
    return dataset

In [ ]:
testing_dataset = testing_dataset_generator(hparas['BATCH_SIZE'], testing_data_generator)

In [ ]:
data = pd.read_pickle('./dataset/testData.pkl')
captions = data['Captions'].values

NUM_TEST = len(captions)
EPOCH_TEST = int(NUM_TEST / hparas['BATCH_SIZE'])

## Inference

In [ ]:
def inference(dataset):
    hidden = text_encoder.initialize_hidden_state()
    sample_size = hparas['BATCH_SIZE']
    sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, hparas['Z_DIM'])).astype(np.float32)
    
    step = 0
    start = time.time()
    for captions, idx in dataset:
        if step > EPOCH_TEST:
            break
        
        fake_image = test_step(captions, sample_seed, hidden)
        step += 1
        for i in range(hparas['BATCH_SIZE']):
            plt.imsave('./inference/demo/inference_{:04d}.jpg'.format(idx[i]), fake_image[i].numpy()*0.5 + 0.5)
            
    print('Time for inference is {:.4f} sec'.format(time.time()-start))

In [ ]:
checkpoint.restore(checkpoint_dir + '/ckpt-20')

In [ ]:
inference(testing_dataset)

## Output score.csv
CAUTION: 
* Please modify GPU setting in <i>inception_score.py</i> if need.
* Please run the below cmd in command line.

In [ ]:
!cd testing
!python inception_score.py ../inference/demo ../score_demo.csv 39 # BATCH_SIZE=39 is available using GTX 1080 Ti (need 9441MB memory)

# Demo - TA80
CAUTION: The code here doesn't work because the inference images aren't provided.

In [ ]:
def visualize(idx):
    fig = plt.figure(figsize=(14, 14))
    
    for count, i in enumerate(idx):
        loc = np.where(i==index)[0][0]
        text = ''
        for word in captions[loc]:
            if id2word_dict[word] != '<PAD>':
                text += id2word_dict[word]
                text += ' '
        print(text)
        
        path = './inference/TA80/inference_{:04d}.jpg'.format(i)
        fake_iamge = plt.imread(path)
        
        plt.subplot(7, 7, count+1)
        plt.imshow(fake_iamge)
        plt.axis('off')

In [ ]:
data = pd.read_pickle('./dataset/testData.pkl')
captions = data['Captions'].values
index = data['ID'].values
random_idx = [23, 216, 224, 413, 713, 859, 876, 974, 1177, 1179, 1241, 2169, 2196, 2237, 
              2356, 2611, 2621, 2786, 2951, 2962, 3145, 3255, 3327, 3639, 3654, 3927, 4262, 
              4321, 4517, 5067, 5147, 5955, 6167, 6216, 6410, 6413, 6579, 6584, 6804, 6988, 
              7049, 7160]

visualize(random_idx)

In [ ]:
DATA_PATH = './inference/TA80/'
img_path = Path(DATA_PATH).glob('*.jpg')
img_path = [str(path.resolve()) for path in img_path]
img_path = np.asarray(img_path)

idx = np.random.randint(len(captions), size=42)
idx.sort()

random_idx = []
for each in idx:
    path = img_path[each].split("/")
    path = path[-1].replace('inference_', '')
    random_idx.append(int(path.replace('.jpg', '')))
    
visualize(random_idx)